# GPU 메모리 관리 유틸리티

이 노트북의 셀들을 복사해서 다른 노트북에서 사용하세요.

## 설치: 이 셀을 다른 노트북에 복사하세요

In [ ]:
import torch
import gc

def clear_gpu_memory(verbose=True):
    """
    GPU 메모리를 완전히 정리합니다.
    """
    if not torch.cuda.is_available():
        if verbose:
            print("CUDA not available. No GPU memory to clear.")
        return None

    # 정리 전 메모리 상태
    before_allocated = torch.cuda.memory_allocated(0) / 1024**2
    before_reserved = torch.cuda.memory_reserved(0) / 1024**2

    if verbose:
        print("=" * 60)
        print("GPU Memory Cleanup")
        print("=" * 60)
        print(f"Before cleanup:")
        print(f"  Allocated: {before_allocated:.2f} MB")
        print(f"  Reserved:  {before_reserved:.2f} MB")

    # 1. Python garbage collection
    gc.collect()

    # 2. PyTorch 캐시 정리
    torch.cuda.empty_cache()

    # 3. 모든 CUDA 스트림 동기화
    torch.cuda.synchronize()

    # 4. 가능하면 메모리 통계 리셋
    try:
        torch.cuda.reset_peak_memory_stats(0)
        torch.cuda.reset_accumulated_memory_stats(0)
    except:
        pass

    # 정리 후 메모리 상태
    after_allocated = torch.cuda.memory_allocated(0) / 1024**2
    after_reserved = torch.cuda.memory_reserved(0) / 1024**2

    freed_allocated = before_allocated - after_allocated
    freed_reserved = before_reserved - after_reserved

    if verbose:
        print(f"\nAfter cleanup:")
        print(f"  Allocated: {after_allocated:.2f} MB (freed: {freed_allocated:.2f} MB)")
        print(f"  Reserved:  {after_reserved:.2f} MB (freed: {freed_reserved:.2f} MB)")
        print("=" * 60)

    return {
        'before': {'allocated': before_allocated, 'reserved': before_reserved},
        'after': {'allocated': after_allocated, 'reserved': after_reserved},
        'freed': {'allocated': freed_allocated, 'reserved': freed_reserved}
    }


def print_gpu_memory(device_id=0):
    """
    현재 GPU 메모리 상태를 출력합니다.
    """
    if not torch.cuda.is_available():
        print("CUDA not available")
        return

    allocated = torch.cuda.memory_allocated(device_id) / 1024**2
    reserved = torch.cuda.memory_reserved(device_id) / 1024**2
    total = torch.cuda.get_device_properties(device_id).total_memory / 1024**2
    free = total - allocated

    print("=" * 60)
    print(f"GPU {device_id}: {torch.cuda.get_device_name(device_id)}")
    print("=" * 60)
    print(f"Total Memory:     {total:.2f} MB")
    print(f"Allocated:        {allocated:.2f} MB ({allocated/total*100:.1f}%)")
    print(f"Reserved:         {reserved:.2f} MB ({reserved/total*100:.1f}%)")
    print(f"Free:             {free:.2f} MB ({free/total*100:.1f}%)")
    print("=" * 60)

    # 경고 메시지
    if allocated / total > 0.9:
        print("⚠️  WARNING: GPU memory usage is very high (>90%)")
    elif allocated / total > 0.8:
        print("⚠️  CAUTION: GPU memory usage is high (>80%)")


print("✓ GPU memory management functions loaded")
print("  - clear_gpu_memory(): GPU 메모리 정리")
print("  - print_gpu_memory(): GPU 메모리 상태 확인")

## 사용 예제

In [ ]:
# GPU 메모리 상태 확인
print_gpu_memory()

In [ ]:
# GPU 메모리 정리
clear_gpu_memory()

## 모델 삭제 및 메모리 정리 예제

In [ ]:
# 모델이 메모리를 차지하고 있다면 삭제
if 'model' in globals():
    del model
    print("✓ Model deleted")

# GPU 메모리 정리
clear_gpu_memory()

## 긴급 메모리 정리 (모든 변수 삭제)

In [ ]:
# ⚠️ 경고: 이 셀은 노트북의 모든 변수를 삭제합니다!
# GPU 메모리가 심각하게 부족할 때만 사용하세요.

import gc

# 주요 객체 삭제
vars_to_delete = ['model', 'optimizer', 'train_loader', 'val_loader', 
                  'train_dataset', 'val_dataset']

for var_name in vars_to_delete:
    if var_name in globals():
        del globals()[var_name]
        print(f"✓ Deleted {var_name}")

# 메모리 정리
gc.collect()
torch.cuda.empty_cache()

print("\n✓ Emergency cleanup completed")
print_gpu_memory()